In [1]:
#!/usr/bin/python3
# Install TensorFlow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers, activations
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import tensorflow_probability as tfp

import numpy as np
import matplotlib.pyplot as plt
# import autokeras as ak
import corner
import os 
import sys
import time
import importlib
import logging
from tqdm import tqdm

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

logging.info("numpy Version is {}".format(np.__version__))
# logging.info("autokeras Version is {}".format(ak.__version__))
logging.info("tensorflow Version is {}".format(tf.keras.__version__))
logging.info("\n")

INFO:root:tensorflow Version is 2.4.0
INFO:root:



1 Physical GPUs, 1 Logical GPU


Ref: https://agustinus.kristia.de/techblog/2016/12/17/conditional-vae/   
Ref: https://keras.io/guides/customizing_what_happens_in_fit/   
Ref: https://keras.io/examples/generative/vae/   
Ref: https://github.com/hagabbar/VItamin/blob/c1ae6dfa27b8ab77193caacddd477fde0dece1c2/Models/VICI_inverse_model.py#L404   
Ref: https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag    

In [2]:
def angle_transform(prediction_array: np.ndarray = 0)-> np.ndarray :
    tmp = np.arctan2(prediction_array[:,0],prediction_array[:,1])
    tmp = np.where(tmp >= 0 , tmp, 2*np.pi+tmp) 
    delta = tmp/np.pi*180 
    
    return delta

def feature_reshape(data,column=2):
    tmp = data.reshape(len(data),column,36)
    tmp = np.array([ element.T for element in tmp])
    return tmp
    

In [3]:
%%time
"""
Load Data
"""
#======================================================#
training_data = np.load("../Data/n1000000_0910_all_flat.npz")
#======================================================#

"""
Stack Data
"""
#======================================================#
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])


"""
Standardization
"""
scaler = StandardScaler()
scaler.fit(data_all)

target = np.column_stack( [training_data["theta23"], training_data["delta"]/180*np.pi ])
# target = target/180*np.pi 

x_train = data_all[:900000]
y_train = target[:900000]
y_train_delta = np.column_stack([np.sin(y_train[:,1]), np.cos(y_train[:,1])]) 
y_train_theta23 = y_train[:,0]



x_test = data_all[900000:]
y_test = target[900000:]
y_test_delta = np.column_stack([np.sin(y_test[:,1]), np.cos(y_test[:,1])]) 
y_test_theta23 = y_test[:,0]


logging.info("X train shape: {}".format(x_train.shape))
logging.info("X test shape: {}".format(x_test.shape))
logging.info("Y train shape: {}".format(y_train.shape))
logging.info("Y test shape: {}".format(y_test.shape))

x_train_poisson = np.random.poisson(x_train)
# x_train_poisson = np.column_stack([x_train_poisson[:,:36]/169.042, x_train_poisson[:,36:72]/787.425, x_train_poisson[:,72:108]/78.6681, x_train_poisson[:,108:144]/334.052])
x_train_poisson = scaler.transform(x_train_poisson) 
x_train_poisson = feature_reshape(x_train_poisson,column=4)

x_test_poisson = np.random.poisson(x_test)
# x_test_poisson = np.column_stack([x_test_poisson[:,:36]/169.042, x_test_poisson[:,36:72]/787.425, x_test_poisson[:,72:108]/78.6681, x_test_poisson[:,108:144]/334.052])
x_test_poisson = scaler.transform(x_test_poisson) 
x_test_poisson = feature_reshape(x_test_poisson,column=4)

logging.info("\n")
logging.info("x_train_poisson shape: {}".format(x_train_poisson.shape))
logging.info("x_test_poisson shape: {}".format(x_test_poisson.shape))
logging.info("\n")

INFO:root:X train shape: (900000, 144)
INFO:root:X test shape: (100000, 144)
INFO:root:Y train shape: (900000, 2)
INFO:root:Y test shape: (100000, 2)
INFO:root:

INFO:root:x_train_poisson shape: (900000, 36, 4)
INFO:root:x_test_poisson shape: (100000, 36, 4)
INFO:root:



CPU times: user 26.2 s, sys: 5.44 s, total: 31.7 s
Wall time: 31.6 s


In [4]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [5]:
latent_dim = 15
logging.info("latent_dim : {}".format(latent_dim))

"""
Encoder 1 (parameter + spectrum)
"""
# parameter
encoder_parameter_inputs = layers.Input(shape=(2,),name = 'encoder_parameter_inputs')
x_parameter = layers.Dense(64, activation="relu", name = 'dense_parameter_1')(encoder_parameter_inputs)
x_parameter = layers.Dense(32, activation="relu", name = 'dense_parameter_2')(x_parameter)
x_parameter = layers.Dense(16, activation="relu", name = 'dense_parameter_3')(x_parameter)

# spectrum
encoder_spectrum_inputs = layers.Input(shape=(36,4),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_2')(x_spectrum)
x_spectrum = layers.Flatten(name = 'flatten_spectrum')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_1')(x_spectrum)

# merged
mergedOut_Encoder_1 = Concatenate()([x_parameter,x_spectrum])

# sampling
z_mean = layers.Dense(latent_dim, name="z_mean")(mergedOut_Encoder_1)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(mergedOut_Encoder_1)
z = Sampling(name = 'Sampling_encoder')([z_mean, z_log_var])

# build model
encoder_1 = keras.Model([encoder_parameter_inputs, encoder_spectrum_inputs], [z_mean, z_log_var, z], name="encoder_1")
encoder_1.summary()

INFO:root:latent_dim : 15


Model: "encoder_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_spectrum_inputs (InputL [(None, 36, 4)]      0                                            
__________________________________________________________________________________________________
encoder_parameter_inputs (Input [(None, 2)]          0                                            
__________________________________________________________________________________________________
conv1d_spectrum_1 (Conv1D)      (None, 27, 5)        205         encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________________
dense_parameter_1 (Dense)       (None, 64)           192         encoder_parameter_inputs[0][0]   
__________________________________________________________________________________________

In [6]:
"""
Encoder 2 (spectrum)
"""
# spectrum
encoder_spectrum_inputs = layers.Input(shape=(36,4),name = 'encoder_spectrum_inputs')
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_1')(encoder_spectrum_inputs)
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_2')(x_spectrum)
x_spectrum = layers.Flatten(name = 'flatten_spectrum')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_1')(x_spectrum)

# sampling
z_mean = layers.Dense(latent_dim, name="z_mean")(x_spectrum)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x_spectrum)
z = Sampling(name = 'Sampling_encoder')([z_mean, z_log_var])

# build model
encoder_2 = keras.Model(encoder_spectrum_inputs, [z_mean, z_log_var, z], name="encoder_2")
encoder_2.summary()

Model: "encoder_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_spectrum_inputs (InputL [(None, 36, 4)]      0                                            
__________________________________________________________________________________________________
conv1d_spectrum_1 (Conv1D)      (None, 27, 5)        205         encoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________________
conv1d_spectrum_2 (Conv1D)      (None, 18, 5)        255         conv1d_spectrum_1[0][0]          
__________________________________________________________________________________________________
flatten_spectrum (Flatten)      (None, 90)           0           conv1d_spectrum_2[0][0]          
__________________________________________________________________________________________

In [7]:
"""
Decoder Model (latent + spectrum)
"""
latent_dim_2 = 2

decoder_latent_inputs = keras.Input(shape=(latent_dim,),name = 'decoder_latent_inputs')
x_latent = layers.Dense(64, activation="relu", name = 'dense_1')(decoder_latent_inputs)
x_latent = layers.Dense(32, activation="relu", name = 'dense_2')(x_latent)
x_latent = layers.Dense(16, activation="relu", name = 'dense_3')(x_latent)


# spectrum
decoder_spectrum_inputs = layers.Input(shape=(36,4),name = 'decoder_spectrum_inputs')
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_1')(decoder_spectrum_inputs)
x_spectrum = layers.Conv1D(filters=5, kernel_size=10,strides=1, activation='relu', name = 'conv1d_spectrum_2')(x_spectrum)
x_spectrum = layers.Flatten(name = 'flatten_spectrum')(x_spectrum)
x_spectrum = layers.Dense(16, activation="relu", name = 'dense_spectrum_1')(x_spectrum)

# merged
mergedOut_Decoder = Concatenate()([x_latent,x_spectrum])


z2_mean = layers.Dense(latent_dim_2, name="z_mean")(mergedOut_Decoder)
z2_log_var = layers.Dense(latent_dim_2, name="z_log_var")(mergedOut_Decoder)
z2 = Sampling(name = 'Sampling_decoder')([z2_mean, z2_log_var])


decoder = keras.Model([decoder_latent_inputs, decoder_spectrum_inputs], [z2_mean, z2_log_var, z2], name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_spectrum_inputs (InputL [(None, 36, 4)]      0                                            
__________________________________________________________________________________________________
decoder_latent_inputs (InputLay [(None, 15)]         0                                            
__________________________________________________________________________________________________
conv1d_spectrum_1 (Conv1D)      (None, 27, 5)        205         decoder_spectrum_inputs[0][0]    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           1024        decoder_latent_inputs[0][0]      
____________________________________________________________________________________________

In [23]:
#Ref: https://keras.io/guides/customizing_what_happens_in_fit/
#Ref: https://keras.io/examples/generative/vae/
#Ref: https://github.com/hagabbar/VItamin/blob/c1ae6dfa27b8ab77193caacddd477fde0dece1c2/Models/VICI_inverse_model.py#L404
class CVAE(keras.Model):
    def __init__(self, encoder1, encoder2, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder1 = encoder1  #(parameter + spectrum)
        self.encoder2 = encoder2  #(spectrum)
        self.decoder = decoder    #(latent + spectrum)
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.reconstruction_loss_theta23_tracker = keras.metrics.Mean(name="loss_theta23_tracker")
        self.reconstruction_loss_delta_tracker = keras.metrics.Mean(name="loss_delta_tracker")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.reconstruction_loss_theta23_tracker,
            self.reconstruction_loss_delta_tracker
                ]

    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            z1_mean, z1_log_var, z1 = self.encoder1(x)     #(parameter + spectrum)
            
            z2_mean, z2_log_var, z2 = self.encoder2(x[1])  #(spectrum)
            
            reconstruction_mean, reconstruction_var, reconstruction = self.decoder([z1, x[1]])      #(latent + spectrum)
            
            
            reconstruction_theta23 = reconstruction[:,0]
            reconstruction_sin = tf.math.sin(reconstruction[:,1])
            reconstruction_sin = tf.reshape(reconstruction_sin, [reconstruction_sin.shape[0],1])
            reconstruction_cos = tf.math.cos(reconstruction[:,1])
            reconstruction_cos = tf.reshape(reconstruction_cos, [reconstruction_cos.shape[0],1])
            reconstruction_delta = tf.concat([reconstruction_sin, reconstruction_cos], 1)
                                                                 
                                                                 
            mse = tf.keras.losses.MeanSquaredError()
            
            reconstruction_loss_theta23 = tf.reduce_mean(
                                            tf.reduce_sum(
                                                        mse(y[0], reconstruction_theta23 )
                                                            )
                                            )
            reconstruction_loss_delta = tf.reduce_mean(
                                            tf.reduce_sum(
                                                        mse(y[1], reconstruction_delta )
                                                            )
                                            )
            

            
            reconstruction_loss = reconstruction_loss_theta23*reconstruction_loss_delta
        
            SMALL_CONSTANT = 1e-12 # necessary to prevent the division by zero in many operations 
            GAUSS_RANGE = 10.0     # Actual range of truncated gaussian when the ramp is 0
            
            # define the r1(z|y) mixture model
            temp_var_r1 = SMALL_CONSTANT + tf.exp(z2_log_var)
            bimix_gauss = tfp.distributions.MultivariateNormalDiag(
                          loc=z2_mean,
                          scale_diag=tf.sqrt(temp_var_r1))
            
            # GET q(z|x,y)
            temp_var_q = SMALL_CONSTANT + tf.exp(z1_log_var)
            mvn_q = tfp.distributions.MultivariateNormalDiag(
                          loc=z1_mean,
                          scale_diag=tf.sqrt(temp_var_q))
            

            
            
            q_samp = mvn_q.sample()  
            log_q_q = mvn_q.log_prob(q_samp)
            log_r1_q = bimix_gauss.log_prob(q_samp)           # evaluate the log prob of r1 at the q samples
            kl_loss = tf.reduce_mean(log_q_q - log_r1_q)      # average over batch
            
        
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reconstruction_loss_theta23_tracker.update_state(reconstruction_loss_theta23)
        self.reconstruction_loss_delta_tracker.update_state(reconstruction_loss_delta)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "theta23_loss": self.reconstruction_loss_theta23_tracker.result(),
            "delta_loss": self.reconstruction_loss_delta_tracker.result(),
        }


In [24]:
"""
Model Training
"""
cvae = CVAE(encoder_1,encoder_2, decoder)
cvae.compile(optimizer=keras.optimizers.Adam())
# vae.compile(optimizer=keras.optimizers.Adadelta())

check_list=[]
csv_logger = CSVLogger("./Training_loss/CVAE_1DCNN_training_log.csv")
check_list.append(csv_logger)



In [ ]:
%%time
cvae.fit( x = [y_train, x_train_poisson],
         y = [y_train_theta23, y_train_delta],
#            validation_split = 0.1,
           batch_size=50,
           epochs=200,
           verbose=1,
#            shuffle = True,
           callbacks=check_list,
         )

Epoch 1/200
11895/18000 [==================>...........] - ETA: 39s - loss: 85.7385 - reconstruction_loss: 14.2303 - kl_loss: 2.1507 - theta23_loss: 125.0923 - delta_loss: 0.1364

In [ ]:
cvae.encoder1.save("./Model/CVAE_1DCNN_encoder_1_test_2.h5")
cvae.encoder2.save("./Model/CVAE_1DCNN_encoder_2_test_2.h5")
cvae.decoder.save("./Model/CVAE_1DCNN_decoder_test_2.h5")

In [ ]:
# cvae_encoder = load_model("./Model/VAE_1DCNN_encoder_v2.h5", compile=False, custom_objects={"Sampling": Sampling})
# cvae_decoder = load_model("./Model/VAE_1DCNN_decoder_v2.h5", compile=False)

cvae_encoder = encoder_2
cvae_decoder = decoder

test_data = np.load('../Data/sample_NuFit0911.npz')

IO_or_NO = 0
N = 1 

if IO_or_NO == 0:
    logging.info("NO")
    logging.info("True point: theta_23 = {:.2f} \delta_cp = {:.2f}".format(test_data['theta23'][0], test_data['delta'][0]))
    true_theta_23, trua_delta = test_data['theta23'][0], test_data['delta'][0]


    #VAE_1DCNN_v2
    data_mid = np.column_stack([test_data["ve_dune"][:,:36],  test_data["vu_dune"][:,:36], test_data["vebar_dune"][:,:36], test_data["vubar_dune"][:,:36]])
    data_NO_mid = data_mid[0]
    logging.info("Test NO Data Shape:{}".format(data_NO_mid.shape))

    data_poisson = np.random.poisson(data_NO_mid, size = (N, len(data_NO_mid)))
    data_poisson = scaler.transform(data_poisson)
    data_poisson = feature_reshape(data_poisson,column=4)
    test_data = data_poisson

    if type(test_data) == list:
        for i in range(len(test_data)):
                logging.info("X ["+str(i)+"] shape {}".format(test_data[i].shape))
    else:
        logging.info("X train/test shape: {}".format(test_data.shape))
    logging.info("\n")
        
else:
    logging.info("IO")
    logging.info("True point: theta_23 = {:.2f} \delta_cp = {:.2f}".format(test_data['theta23'][1], test_data['delta'][1]))
    true_theta_23, trua_delta = test_data['theta23'][1], test_data['delta'][1]


    #VAE_1DCNN_v2
    data_mid = np.column_stack([test_data["ve_dune"][:,:36],  test_data["vu_dune"][:,:36], test_data["vebar_dune"][:,:36], test_data["vubar_dune"][:,:36]])
    data_IO_mid = data_mid[1]
    logging.info("Test IO Data Shape:{}".format(data_IO_mid.shape))

    data_poisson = np.random.poisson(data_IO_mid, size = (N, len(data_IO_mid)))
    data_poisson = scaler.transform(data_poisson)
    data_poisson = feature_reshape(data_poisson,column=4)
    test_data = data_poisson

    if type(test_data) == list:
        for i in range(len(test_data)):
                logging.info("X ["+str(i)+"] shape {}".format(test_data[i].shape))
    else:
        logging.info("X train/test shape: {}".format(test_data.shape))
    logging.info("\n")

In [18]:
# %%time
# #VAE

# Z3 = []
# prediction = []
# Z = cvae_encoder.predict(data_poisson)

# for i in range(100):
#     z = Sampling()([Z[0], Z[1]])
#     Z3.append(z.numpy())

# Z3 = np.array(Z3)
# Z3 = Z3.reshape(Z3.shape[0], Z3.shape[2])

# logging.info(Z3.shape)

In [13]:
%%time
#CVAE

n_test= 10000

Z = cvae_encoder.predict(data_poisson)
Z3 = []
data_poisson_array = data_poisson

for i in range(n_test):
    z = Sampling()([Z[0], Z[1]])
    Z3.append(z.numpy())
    
for i in range(n_test-1):   
    data_poisson_array = np.concatenate([data_poisson_array,data_poisson])

Z3 = np.array(Z3)
Z3 = Z3.reshape(Z3.shape[0], Z3.shape[2])

logging.info(Z3.shape)
logging.info(data_poisson_array.shape)

INFO:root:(10000, 15)
INFO:root:(10000, 36, 4)


CPU times: user 23.7 s, sys: 5.86 s, total: 29.6 s
Wall time: 44.9 s


In [17]:
%%time
prediction = cvae_decoder.predict([Z3, data_poisson_array])[2]
input_theta23 = np.array(prediction[:,0])
# input_delta = np.array(prediction[:,1]/np.pi*180 )
tmp = np.where(prediction[:,1] >= 0 , prediction[:,1], 2*np.pi+prediction[:,1]) 
input_delta = tmp/np.pi*180 
# input_theta23 = np.array(prediction[0])
# input_delta = np.array(angle_transform(prediction[1]))

bins_theta23_globes = 100
bins_delta_globes = 100

# true_theta_23, trua_delta = y_test_theta23[index], angle_transform(np.array([y_test_delta[index]]))[0]

CPU times: user 452 ms, sys: 55.7 ms, total: 508 ms
Wall time: 543 ms


In [ ]:
plt.hist(input_theta23,orientation="horizontal",density=1, bins=bins_delta_globes)
plt.show()

In [ ]:
%%time

likeliregion, xedges, yedges = np.histogram2d(input_theta23, input_delta, bins = [bins_theta23_globes, bins_delta_globes])
likeliregion = likeliregion.T

max_poi = np.where(likeliregion == likeliregion.max())
print("Maximum: theta23 {}, delta {} ".format(xedges[max_poi[1]],yedges[max_poi[0]]))

In [ ]:
%%time

fig, ax = plt.subplots(1,1, figsize=(15,15))

#=========================== Upper Left Corner
plot_axis = plt.subplot(2,2,1)
"""
ML
"""
plot_axis.hist(input_theta23,density=1,bins=bins_theta23_globes)
if IO_or_NO == 0:
    plot_axis.set_xlim((44,54))
elif IO_or_NO == 1:
    plot_axis.set_xlim((44,54))
plot_axis.tick_params(axis='x', labelsize=25)
plot_axis.tick_params(axis='y', labelsize=25)
plot_axis.set_ylabel(r'density', fontsize=30)
#===========================




#=========================== Down Right Corner
plot_axis = plt.subplot(2,2,4)
"""
ML
"""
plot_axis.hist(input_delta,orientation="horizontal",density=1, bins=bins_delta_globes)
# if IO_or_NO == 0:
#     plot_axis.set_ylim((145,245))
# elif IO_or_NO == 1:
#     plot_axis.set_ylim((236, 336))
plot_axis.tick_params(axis='x', labelsize=25)
plot_axis.tick_params(axis='y', labelsize=25)
plot_axis.set_xlabel(r'density', fontsize=30)
#===========================


#=========================== Down Left Corner
plot_axis = plt.subplot(2,2,3)

# """
# ML (Poisson)
# """
corner.hist2d(input_theta23, input_delta,
                    levels=(0.68,),
                    scale_hist=True,
                    plot_datapoints=False,
                    color='green',
                    labels= ["$\\theta_{23} $($^\circ$)", "$\delta_{cp} $($^\circ$)"],
                    range=[[true_theta_23-5,true_theta_23+5], [trua_delta-50, trua_delta+50]],
                    plot_contours = True,
                    plot_density = False,
                    fontsize=30,
#                     bins = [bins_theta23_globes, bins_delta_globes],
                    label_kwargs={"fontsize": 30},
                    smooth=True,
#                     quantiles=[0.16, 0.5, 0.84],
#                     show_titles=True,
                   )


corner.hist2d(input_theta23, input_delta,
                    levels=(0.95,),
                    scale_hist=True,
                    plot_datapoints=False,
                    color='red',
                    labels= ["$\\theta_{23} $($^\circ$)", "$\delta_{cp} $($^\circ$)"],
                    range=[[true_theta_23-5,true_theta_23+5], [trua_delta-50, trua_delta+50]],
                    plot_contours = True,
                    plot_density = False,
                    fontsize=30,
#                     bins = [bins_theta23_globes, bins_delta_globes],
                    label_kwargs={"fontsize": 30},
                    smooth=True
                   )





if IO_or_NO == 0:
    plot_axis.scatter(true_theta_23, trua_delta, marker="d", c="k", s=10, label = "True Point: $\\theta_{23}$=%.2f, $\delta_{cp}$=%.2f" %(true_theta_23, trua_delta))
else:
    plot_axis.scatter(true_theta_23, trua_delta, marker="d", c="k", s=10, label = "True Point: $\\theta_{23}$=%.2f, $\delta_{cp}$=%.2f" %(true_theta_23, trua_delta))

plot_axis.scatter(xedges[max_poi[1]],yedges[max_poi[0]+1], marker="s", c="r", s=10, label = "Maximum Point: $\\theta_{23}$=%.2f, $\delta_{cp}$=%.2f" %(xedges[max_poi[1]][0], yedges[max_poi[0]+1][0]))

    
# """
# globes
# """
# DU = plot_axis.contour(X0, Y0, Z0, 0, colors='green', linestyles="--", linewidths=1 )
# DU.collections[0].set_label("GLOBES (1$\sigma$)")

# DU_IO = plot_axis.contour(X1, Y1, Z1, 0, colors='green', linestyles="--", linewidths=1 )
# DU_IO.collections[0].set_label("GLOBES (1$\sigma$)")


# """
# ML (Poisson)
# """
# CS_1_sigma = plt.contour(xaxis, yaxis, one_sigma_region_boundary, 0, colors='green', linestyles="-", linewidths=1)
# CS_2_sigma = plt.contour(xaxis, yaxis, two_sigma_region_boundary, 0, colors='red', linestyles="-", linewidths=1)
# CS_1_sigma.collections[0].set_label("ML Estimate (1$\sigma$)")
# CS_2_sigma.collections[0].set_label("ML Estimate (2$\sigma$)")


plot_axis.set_xlabel(r'$\theta_{23} $($^\circ$)', fontsize=30)
plot_axis.set_ylabel(r'$\delta_{cp} $($^\circ$)', fontsize=30)
# if IO_or_NO == 0:
#     plot_axis.set_ylim((145,245))
#     plot_axis.set_xlim((44,54))
# elif IO_or_NO == 1:
#     plot_axis.set_ylim((236, 336))
#     plot_axis.set_xlim((44,54))
# else:
#     raise ValueError("aaaa")
    
    
plot_axis.tick_params(axis='x', labelsize=25)
plot_axis.tick_params(axis='y', labelsize=25)
#===========================
                              
    

#=========================== Whole Figure Setting

if IO_or_NO == 0:
    plt.text(x=54.5,y=355, s="Normal Ordering", fontsize=25 )
#     plt.text(x=54.5,y=340, s= str(index)+" Training", fontsize=25 )
    
elif IO_or_NO == 1:
    plt.text(x=54.5,y=440, s="Inverse Ordering", fontsize=25 )
#     plt.text(x=54.5,y=430, s= str(index)+" Training", fontsize=25 ) 
    
# plt.subplots_adjust(wspace=0.15, hspace=0.01)
plt.legend(bbox_to_anchor=(2.3, 1.7), ncol=1,fontsize=20, markerscale=4, edgecolor = "w",fancybox=False, framealpha=0)
# plt.savefig("./Plots/"+str(experiment)+"_"+str("to_5GeV")+"_"+str("IO_")+str(index)+".pdf", transparent=True, bbox_inches='tight') 
plt.show()